<a href="https://colab.research.google.com/github/scrannah/dnnls_final_project/blob/master/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
rm -rf /content/dnnls_final_project


In [32]:
%cd /content


/content


In [33]:

!git clone https://github.com/scrannah/dnnls_final_project.git
%cd /content/dnnls_final_project


Cloning into 'dnnls_final_project'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 249 (delta 140), reused 173 (delta 67), pack-reused 0 (from 0)
Receiving objects: 100% (249/249), 34.36 KiB | 8.59 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/dnnls_final_project


In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
# @title Importing the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
import os

from datasets import load_dataset

from datasets.fingerprint import random
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.transforms.functional as FT
import torch.nn.functional as F
from torchmetrics.image import StructuralSimilarityIndexMeasure
from nltk.translate.bleu_score import sentence_bleu

from transformers import BertTokenizer
import gc

import textwrap
import sys
import os

abspath = r".\dnnls_final_project"
sys.path.append(os.path.abspath(abspath ))

# put src imports here models training utils etc
from src.attention.attention import Attention

from src.dataloaders.sp_dataset import SequencePredictionDataset
from src.dataloaders.tag_extraction import parse_gdi_text
from src.dataloaders.text_dataset import TextTaskDataset
from src.dataloaders.vae_dataset import AutoEncoderTaskDataset

from src.encoders.visual_autoencoder import Backbone, VisualEncoder, VisualDecoder, VisualAutoencoder
from src.encoders.text_autoencoder import EncoderLSTM, DecoderLSTM, Seq2SeqLSTM

from src.model.sequence_predictor import SequencePredictor

from src.train.train_sequence_predictor import train_sequence_predictor
from src.train.train_visual_autoencoder import train_visual_autoencoder

from src.utils.checkpoints import save_checkpoint_to_drive, load_checkpoint_from_drive
from src.utils.training_utils import init_weights, validation
from src.utils.token_generate import generate


In [36]:
att = Attention(12)
print(att) # test to see if attention imports

Attention(
  (attn): Linear(in_features=12, out_features=1, bias=True)
  (softmax): Softmax(dim=1)
)


In [37]:
# @title Variables and initial setup
torch.cuda.empty_cache()
gc.collect()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_EPOCHS = 5
emb_dim = 16
latent_dim = 16
num_layers = 1
dropout = True

In [38]:
#  @title Image functions


# Need to add loss curves

# Image denormalisation
#  ax.imshow(image.permute(1, 2, 0))

In [39]:
# @title Dataset loading/creation

# Loading the dataset
train_dataset = load_dataset("daniel3303/StoryReasoning", split="train")
test_dataset = load_dataset("daniel3303/StoryReasoning", split="test")

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_indices, val_indices = random_split(
    range(len(train_dataset)),
    [train_size, val_size]) # split the database universally here, avoids data leakage
    # for a range of the len of train set, split into these indices

# For the Sequence prediction task
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased",  padding=True, truncation=True)
sp_train_dataset = SequencePredictionDataset(train_dataset, tokenizer) # Instantiate the train dataset
sp_test_dataset = SequencePredictionDataset(test_dataset, tokenizer) # Instantiate the test dataset

# Instantiate the dataloaders
sp_train_subset = Subset(sp_train_dataset, train_indices)
train_dataloader = DataLoader(sp_train_subset, batch_size=8, shuffle=True)
# We will use the validation set to visualize the progress.
sp_val_subset = Subset(sp_train_dataset, val_indices)
val_dataloader = DataLoader(sp_val_subset, batch_size=4, shuffle=False)

test_dataloader = DataLoader(sp_test_dataset, batch_size=4, shuffle=False)

# Text dataset
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased",  padding=True, truncation=True)
text_dataset = TextTaskDataset(train_dataset)
text_test_dataset = TextTaskDataset(test_dataset)

# Universal split applied HERE (use the same indices)
text_train_subset = Subset(text_dataset, train_indices)
text_val_subset   = Subset(text_dataset, val_indices)

text_train_dataloader = DataLoader(text_train_subset, batch_size=4, shuffle=True)
text_val_dataloader   = DataLoader(text_val_subset, batch_size=4, shuffle=False)
text_test_dataloader  = DataLoader(text_test_dataset, batch_size=4, shuffle=False)

# Image dataset
autoencoder_train_dataset = AutoEncoderTaskDataset(train_dataset)
autoencoder_test_dataset = AutoEncoderTaskDataset(test_dataset)
ae_train_subset = Subset(autoencoder_train_dataset, train_indices)
ae_val_subset   = Subset(autoencoder_train_dataset, val_indices)

autoencoder_train_dataloader = DataLoader(ae_train_subset, batch_size=4, shuffle=True,
                                    collate_fn=lambda batch: torch.stack(batch, dim=0))
autoencoder_test_dataloader = DataLoader(autoencoder_test_dataset, batch_size=4, shuffle=False,
                                    collate_fn=lambda batch: torch.stack(batch, dim=0))
autoencoder_val_dataloader = DataLoader(ae_val_subset, batch_size=4, shuffle=False,
                                    collate_fn=lambda batch: torch.stack(batch, dim=0))

In [40]:
# @title Initialising models and setup

# Initializing the NLP models
encoder = EncoderLSTM(tokenizer.vocab_size, emb_dim, latent_dim, num_layers, dropout).to(device)
decoder = DecoderLSTM(tokenizer.vocab_size, emb_dim, latent_dim, num_layers, dropout).to(device)
text_autoencoder = Seq2SeqLSTM(encoder, decoder).to(device)
text_autoencoder, _, _, _ = load_checkpoint_from_drive(text_autoencoder, None, filename='text_autoencoder.pth')

total_params = sum(p.numel() for p in text_autoencoder.parameters())
print(f"Total parameters (Not trainable): {total_params}")
# Deactivating training from this model for efficiency
for param in text_autoencoder.parameters():
    param.requires_grad = False


# Initializing visual models
visual_autoencoder = VisualAutoencoder(latent_dim=16).to(device)
# visual_autoencoder.apply(init_weights)

# IMPORTANT: Load checkpoint WITH optimizer because we are continuing training
visual_ae_optimizer = torch.optim.Adam(
    (p for p in visual_autoencoder.parameters() if p.requires_grad),
    lr=1e-4
)

visual_autoencoder, _, epoch_loaded, loss_loaded = load_checkpoint_from_drive(
    visual_autoencoder,
    None,
    filename="visual_autoencoder.pth"
)

total_params = sum(p.numel() for p in visual_autoencoder.parameters() if p.requires_grad)
print(f"Total trainable parameters in visual autoencoder: {total_params}")
print(f"Visual Autoencoder loaded from epoch {epoch_loaded} with loss {loss_loaded:.4f}")


# We put all the sizes the same, not ideal as well
sequence_predictor = SequencePredictor(
    visual_autoencoder,
    text_autoencoder,
    latent_dim,
    latent_dim
).to(device)

total_params = sum(p.numel() for p in sequence_predictor.parameters() if p.requires_grad)
print(f"Total trainable parameters in the whole model: {total_params}")

total_params = sum(p.numel() for p in sequence_predictor.parameters())
print(f"Total parameters: {total_params}")


# Training tools
criterion_images = nn.L1Loss()
criterion_ctx = nn.MSELoss()
criterion_text = nn.CrossEntropyLoss(ignore_index=tokenizer.convert_tokens_to_ids(tokenizer.pad_token))

# Optimizer for SEQUENCE PREDICTOR created AFTER model definition
optimizer = torch.optim.Adam(
    (p for p in sequence_predictor.parameters() if p.requires_grad),
    lr=0.001
)


Checkpoint loaded from: /content/gdrive/MyDrive/DL_Checkpoints/text_autoencoder.pth (epoch 15)
Total parameters (Not trainable): 1499930
Checkpoint loaded from: /content/gdrive/MyDrive/DL_Checkpoints/visual_autoencoder.pth (epoch 0)
Total trainable parameters in visual autoencoder: 503539
Visual Autoencoder loaded from epoch 0 with loss 0.0870
Total trainable parameters in the whole model: 507028
Total parameters: 2006958


In [41]:
# @title Sequence predictor training sequence
losses = []
all_epoch_losses = []
all_train_mse = []
all_train_perplexity = []
all_train_bleu = []
all_train_crossmodal = []
all_train_ssim = []

all_val_mse = []
all_val_perplexity = []
all_val_bleu = []
all_val_crossmodal = []
all_val_ssim = []



for epoch in range(N_EPOCHS):

    metrics = train_sequence_predictor(
        model=sequence_predictor,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        optimizer=optimizer,
        criterion_images=criterion_images,
        criterion_ctx=criterion_ctx,
        criterion_text=criterion_text,
        tokenizer=tokenizer,
        device=device,
        num_epochs=1
    )

    all_epoch_losses.append(metrics["epoch_losses"][0])

    all_train_mse.append(metrics["train_mse"][0])
    all_train_perplexity.append(metrics["train_perplexity"][0])
    all_train_bleu.append(metrics["train_bleu"][0])
    all_train_crossmodal.append(metrics["train_crossmodal"][0])
    all_train_ssim.append(metrics["train_ssim"][0])

    all_val_mse.append(metrics["val_mse"][0])
    all_val_perplexity.append(metrics["val_perplexity"][0])
    all_val_bleu.append(metrics["val_bleu"][0])
    all_val_crossmodal.append(metrics["val_crossmodal"][0])
    all_val_ssim.append(metrics["val_ssim"][0])

    if epoch % 5 == 0:
        save_checkpoint_to_drive(
            sequence_predictor,
            optimizer,
            epoch,
            all_epoch_losses[-1],
            filename=f"sequence_predictor.pth"
        )


Validation on training dataset
----------------


NameError: name 'sequence_predictor' is not defined

In [ ]:
# @title Image autoencoder training

for epoch in range(N_EPOCHS):

    epoch_losses = train_visual_autoencoder(
        model=visual_autoencoder,
        dataloader=autoencoder_train_dataloader,
        optimizer=visual_ae_optimizer,
        criterion=criterion_images,
        device=device,
        num_epochs=1
    )

    if epoch % 5 == 0:
        save_checkpoint_to_drive(
            visual_autoencoder,
            visual_ae_optimizer,
            epoch,
            epoch_losses[-1],
            filename=f"visual_autoencoder.pth"
        )


In [ ]:
# @title SP METRIC PLOTTING

# Training Loss Curve
plt.plot(all_epoch_losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Time")
plt.show()

# Training SSIM
plt.plot(all_train_ssim)
plt.xlabel("Epoch")
plt.ylabel("SSIM")
plt.title("Training SSIM Over Epochs")
plt.show()

# Training BLEU
plt.plot(all_train_bleu)
plt.xlabel("Epoch")
plt.ylabel("BLEU")
plt.title("Training BLEU Over Epochs")
plt.show()

# Training MSE
plt.plot(all_train_mse)
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.title("Training MSE Over Epochs")
plt.show()

# Training Perplexity
plt.plot(all_train_perplexity)
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Training Perplexity Over Epochs")
plt.show()

# Training Cross-Modal Similarity
plt.plot(all_train_crossmodal)
plt.xlabel("Epoch")
plt.ylabel("Cross-modal similarity")
plt.title("Training Cross-modal Similarity Over Epochs")
plt.show()

# Validation SSIM
plt.plot(all_val_ssim)
plt.xlabel("Epoch")
plt.ylabel("SSIM")
plt.title("Validation SSIM Over Epochs")
plt.show()

# Validation BLEU
plt.plot(all_val_bleu)
plt.xlabel("Epoch")
plt.ylabel("BLEU")
plt.title("Validation BLEU Over Epochs")
plt.show()

# Validation MSE
plt.plot(all_val_mse)
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.title("Validation MSE Over Epochs")
plt.show()

# Validation Perplexity
plt.plot(all_val_perplexity)
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Validation Perplexity Over Epochs")
plt.show()

# Validation Cross-Modal Similarity
plt.plot(all_val_crossmodal)
plt.xlabel("Epoch")
plt.ylabel("Cross-modal similarity")
plt.title("Validation Cross-modal Similarity Over Epochs")
plt.show()
